# Mask Scorer

## Synopsis

python2 MaskScorer.py -t task-type<br/>
                      --refDir reference-directory<br/>
                      -r reference-file<br/>
                      -x index-file<br/>
                      -s system-output-file<br/>
                      --outRoot output-directory<br/>
                      [options]

## Description

The mask scorer calculates performance scores that measure the accuracy of the system output masks to their corresponding reference masks. The script produces output files in the form of pipe-separated ("|") CSV files, one containing scores at the trial level (\*-mask_scores-perimage.csv), another containing an average of the metrics in the first CSV (\*-mask_score.csv), and a third containing a list of manipulations that were evaluated corresponding to the journal in each probe (\*-journalResults.csv). If the -html option is selected, the script will generate a detailed HTML file for the mask region performance results.

The mask scorer takes the following steps to score a mask:

1. Reads in the reference and system masks.
2. Binarizes the reference mask.
3. Generates the no-score zone.
4. Computes metrics 

If the script aborts due to an error, the error will be written to standard out, and the script will exit with a status of 1.

## Reading the Masks

The system mask is read in as a single-layer grayscale png. (Note that the validator will fail the mask if it is not.) The reference mask is read in as a single-layer JPEG2000 containing bit values corresponding to localizable manipulations. If the evaluation is against color reference masks where mutually exclusive regions are RGB color-coded and different regions are easily distinguishable, the mask is read in as a three-channel (BGR-channel) png mask.

## Binarizing the Reference Mask

In order to score the system output mask against the reference mask, the reference mask also needs to be converted to a single-channel mask. The obvious way to do this would be to treat each of the regions with bit values (color regions for the color masks) as manipulated regions and to score accordingly.

If selective scoring is utilized (via the -qm option), only regions that match the query will be scored against. The rest will be treated as no-score regions, to be discussed below.

## Generating No-Score Zones

No-score zones are generated by differencing the erosion and dilation of the ground truth positive pixels in the reference mask. The difference between the two is called the boundary no-score zone.

If selective scoring is utilized (via the -qm option), the mask scorer also generates the distraction no-score zone by dilating the positive pixels of the irrelevant regions in the reference mask. This number need not be the same as the dilation parameter used to generate the boundary no-score zone.

An additional system-generated no-score zone may be used by the performer. The performer specifies a particular pixel value to treat as a no-score zone. This no-score zone is joined with the aforementioned no-score zones.

## Metrics

The system computes the following metrics:
 * Matthews Correlation Coefficient (MCC)
 * Nimble Mask Metric (NMM)
 * Weighted L1 Loss: Binary (BWL1) and Grayscale (GWL1)
 * Area Under the ROC Curve (AUC)

The set of metrics returned will be the ones with the largest MCC. These are called the "optimum" metrics. Additionally specifying a threshold for all images to be binarized via the --sbin option will yield the "maximum" and "actual" metrics: the maximum metrics are defined as the metric values that yield the largest average MCC for a fixed threshold across all images; the actual metrics are defined as the metric values at the binarization threshold chosen by the performer.

Averages of the metrics will be recorded across all probes. The average and standard deviation of the optimum thresholds will also be recorded.

### Notation

The following notation is used in discussing the metrics:
 * $gt$ refers to the binarized reference mask
 * $sys_{\theta}$ refers to the system output mask binarized by threshold $\theta$
   * $sys_{*}$ refers to the unbinarized grayscale system output mask
 * $wts$ is a matrix of 1's and 0's, which denote the pixels scored and exempt by the system respectively. It is generated by a difference in the erosion and dilation of the manipulated area of $gt$.
 * $TP$, $TN$, $FP$, and $FN$ are functions of $gt$, $sys_{\theta}$, and $wts$. All four are measures derived from the confusion matrix. Pixels for which the weights matrix is equal to 0 are not counted in the confusion measures. These will also be reported in the perimage output file:
   * $TP$ refers to the total number of True Positives, pixels where $gt$ is positive and $sys_{\theta}$ is thresholded positive (manipulated)
   * $TN$ refers to the total number of True Negatives, pixels where $gt$ is negative and $sys_{\theta}$ is positive (not manipulated)
   * $FP$ refers to the total number of False Positives, pixels where $gt$ is negative but $sys_{\theta}$ is positive.
   * $FN$ refers to the total number of False Negatives, pixels where $gt$ is positive but $sys_{\theta}$ is negative.

### Matthews Correlation Coefficient (MCC)
\begin{equation*}
MCC(gt,sys_{\theta}) = \frac{TP*TN - FP*FN}{\sqrt{(TP+FP)(TP+FN)(TN+FP)(TN+FN)}}
\end{equation*}

An MCC of 1 denotes perfect correlation, an MCC of 0 denotes no correlation at all, and an MCC of -1 denotes perfect anti-correlation. If any of the sums in the denominator equals 0, the MCC is taken to be 0 by convention.

### Nimble Mask Metric (NMM)
\begin{equation*}
NMM(gt,sys_{\theta},wts,c)=\max{\left(\frac{TP - FN - FP}{\sum\limits_{px\in gt}wts(px)},c\right)}
\end{equation*}

$\Sigma_{px \in GT}$ refers to the sum over the pixels in the ground truth that are marked black. $c$ denotes a minimum cutoff value for the scoring to have any meaning; by default, $c=-1$.

### Weighted L1 Loss (BWL1 and GWL1)

A Weighted L1 of 0 denotes perfect or near perfect match up to variation within the weights that are 0; 1 denotes perfect mismatch. $(FP+FN)_{weights > 0}$ refers to the total number of $FP$ and $FN$ pixels where weights are greater than 0.

The Weighted L1 is applied separately to the original grayscale system output and the binarized mask, producing the grayscale Weighted L1 (GWL1) and binarized Weighted L1 (BWL1) metrics respectively. In the case of the original grayscale, the value is summed over the weighted difference in pixel intensity.

\begin{equation*}
BWL1(gt,sys_{\theta},wts)=\frac{(FP+FN)_{wts > 0}}{\sum\limits_{px\in gt}wts(px)}
\end{equation*}
\begin{equation*}
GWL1(gt,sys_{*},wts)=\frac{\sum\limits_{px\in gt} \left|gt(px) - sys_{*}(px)\right|wts(px)}{\sum\limits_{px\in gt}wts(px)}
\end{equation*}

### Area Under Curve (AUC)

The possibility of multiple thresholds giving rise to a set of varying confusion measures gives rise to the receiver operating characteristic (ROC). The ROC curve measures a relation between the true positive rate ($TPR$) and false positive rate ($FPR$), defined as follows:

\begin{equation*}
TPR = \frac{TP}{TP + FN}
\end{equation*}

\begin{equation*}
FPR = \frac{FP}{FP + TN}
\end{equation*}

The area under the ROC curve (AUC) is a measure for the accuracy of the system.


\begin{equation*}
AUC(gt,sys_{*},wts) = \sum\limits_{\theta_{n} \in \{\theta_{0},\ldots,\theta_{N-1}\}} \left(TPR(gt,sys_{\theta_{n+1}},wts) + TPR(gt,sys_{\theta_{n}},wts)\right)\left(FPR(gt,sys_{\theta_{n+1}},wts) - FPR(gt,sys_{\theta_{n}},wts)\right)
\end{equation*}

## ROC Curves

A pixel-weighted average ROC curve and probe-weighted average ROC curve will be recorded and defined across all thresholds relevant to all system masks. For a given threshold $\theta$, the true positive rate (TPR) for the ROC curves are as follows:

\begin{equation*}
TPR_{pixel}(\theta) = \frac{\sum_{probes}TP_{\theta}}{\sum_{probes}\left(TP_{\theta} + FN_{\theta}\right)}
\end{equation*}

\begin{equation*}
TPR_{probe}(\theta) = \frac{1}{|probes|}\sum_{probes}\frac{TP_{\theta}}{TP_{\theta} + FN_{\theta}}
\end{equation*}

The false postive rates (FPR) are defined similarly. The pixel weighted average and probe weighted average ROC curves are drawn from the pairs of (FPR,TPR) points.

## Options

The command-line options for the mask scorer can be categorized as follows:

### Task Type Options

-t --task [manipulation, splice]

  * Specify the task type for evaluation (default = manipulation)

### Input Options

All CSV files passed to the Mask Scorer must contain headers and must have their rows separated by pipe characters ('|'). Fields and values in the CSV should <i>not</i> be enclosed in quotes ( ' or " ) if possible (e.g. entries 'foo', an empty field, and 'bar', in that order, should look like this in the csv: foo||bar). Additional specifications for the index and system output files can be found in the ValidatorNotebook.html file under the Validator directory.

--refDir

  * Specify the reference and index data path (e.g. "/NC2016_Test0601") (default = .)


-r --inRef

  * Specify the reference CSV file within refDir that contains the ground-truth information and metadata about each image. Key fields are TaskID, ProbeFileID, ProbeFileName, and ProbeMaskFileName, and if scoring on the 'splice' task, DonorFileID, DonorFileName, and DonorMaskFileName as well. Often the File ID's for the Probe and Donor will be the same as the file names, minus the extension. Additional fields, especially metadata pertaining to the ground-truth manipulation, may be included.

-x --inIndex

  * Define the index CSV file within refDir. The index file contains the TaskID, ProbeFileID, ProbeFileName, ProbeWidth, and ProbeHeight fields, and if scoring on the splice task, the DonorFileID, DonorFileName, DonorWidth, and DonorHeight fields as well. No additional fields are permitted for the index file.

--sysDir

  * Specify the system output data path, for example "mysysoutput/" (default = .) 


-s --inSys

  * Specify the CSV file of the system performance results formatted according to NC2016 specification. The file must contain the ProbeFileID, ConfidenceScore, and ProbeOutputMaskFileName fields, in that order, and if scoring on the splice task, the ProbeFileID, DonorFileID, ConfidenceScore, ProbeOutputMaskFileName, and DonorOutputMaskFileName fields, in that order. The ProbeOutputMaskFileNames and DonorOutputMaskFileNames (where relevant) should be directory strings relative to the location of the system performance CSV.

--sbin

  * Evaluate the system output masks as binarized masks with a numeric threshold in the interval [0,255]. Choosing -1 will forego this option. (default = -1)

--color

  * Evaluate the system output against colorized referenced masks. Individual regions in the colorized masks are identifiable by region and do not intersect.

### Output Options

--outRoot

  * Specify the report output path. The file name will take from the name of the For example, if you specify "--outRoot test" for a submission NIST_001, you will find the aggregate score report "NIST_001-mask_score.csv" and the per-image report "NIST_001-mask_scores_perimage.csv" in the "test" folder.

--outMeta
  * Save the CSV file with the system output with minimal metadata. This is a separate file than the normal outputs.

--outAllMeta
  * Save the CSV file with the system output with all available metadata. This is a separate file than the normal outputs.

### Scoring Options

--eks

  * Erosion kernel size. (number must be odd; default = 15)
  
--dks

  * Dilation kernel size. (number must be odd; default = 11)

--ntdks

  * Non-target dilation kernel for regions that the user does not want scored. (number must be odd; default = 15)

--nspx

  * Set a pixel value in the system output mask to be a no-score region. The pixel value must be in the interval [0,255]. -1 indicates that no particular pixel value will be chosen to be the no-score zone. (default = -1)

-k kernel

  * The shape of the kernel to be used, for both erosion and dilation. Choose from 'box','disc','diamond','gaussian', or 'line'. The default kernel is 'box'.
  
-xF indexFilter

  * Filter scoring to only files that are present in the index file. This option permits scoring to select index files for the purpose of testing, and may accept system outputs that have not passed the validator.

--speedup

  * Run mask evaluation with a sped-up evaluator.
  
-p processors

  * Run mask evaluation with multiple processors to speed up computation further. (default = 1)

### Performance Evaluation by Query

This option allows the user to evaluate their algorithm performance on either subsets or partitions of the data based on the specified queries and query options. The reference and index CSV files contain a list of factors (e.g., ProbePostProcessed|DonorPostProcessed|ManipulationQuality|IsManipulationTypeRemoval|...). Selecting none of the following factors will output a single report table (CSV) over the entire computed dataset.

-q query
 * Evaluate algorithm performance on a partitioned dataset using multiple factor queries, one at a time (e.g. "Collection==['NC2017'] & Purpose==['add','remove']" will average over the rows that fit this criterion for one queried average, but "Collection==['NC2017']" "Purpose==['add','remove']" will average over the first and then the second independently for two queried averages). The option generates N report tables (CSV), one for each query.
   * Syntax: -q "query1" "query2" "query3" ...
   - The syntax is the same as Pandas' query syntax. Please see the detailed query rule in the website: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-query.
   
   Examples:
   
   % -q "Collection==['Nimble-SCI']" => 1 query
   
   % -q "Collection==['Nimble-SCI'] and PostProcessing=['rescale']" => 1 query
   
   % -q "Collection==['Nimble-SCI','Nimble-WEB']" "PostProcessing=['rescale']" "200<ProbeWidth<=3000" => 3 queries

-qp queryPartition
 * Uses one factor query to evaluate algorithm performance on a partitioned dataset through its individual sub-queries (e.g. "Collection==['NC2017'] & Purpose==['add','remove']" will average over "Collection==['NC2017'] & Purpose==['add']" and "Collection==['NC2017'] & Purpose==['remove']" for a total of two queried averages). This option generates a single report table (CSV) that contains M partition results, one result for each query.
   * Syntax: -qp "query"
   
   Examples:
   
   % -qp "Collection==['Nimble-SCI']" => 1 partition
   
   % -qp "Collection==['Nimble-SCI','Nimble-WEB'] & PostProcessing=['rescale']" => 2 partitions
   
   % -qp "Collection==['Nimble-SCI','Nimble-WEB'] & PostProcessing=['rescale','noise']" => 4 partitions
   
-qm queryManipulation
 * Filters the dataset before scoring takes place for some number of queries. It is functionally similar to the -q query option. The option generates M report tables (CSV), one for each query.
   * Syntax: -qm "query1" "query2" "query3" ...
   - Like the -q option, the syntax is the same as Pandas' query syntax.
   
   Examples:
   
   % -qm "Purpose==['remove']" => 1 query
   
   % -qm "Operation==['PasteSplice']" "Operation==['FillContentAwareFill']" => 2 query
   
   % -qm "Purpose==['remove']" "Purpose==['add']" "Purpose==['splice']"=> 3 queries

### Report Options

-v verbose

  * Control print output. Select 1 to print all non-error related output and 0 to suppress all print output (bar argument-parsing errors).
  
--precision

  * The number of digits to round computed scores, (e.g. a score of 0.3333333333333... will round to 0.33333 for a precision of 5), (default = 16).

-html

  * Output the report to HTML files to visualize scoring.
  
--optOut

  * Evaluate algorithm performance on trials where the IsOptOut value is not 'Y'.
  
--displayScoredOnly

  * Display only the data for which a localized score could be generated. This will exclude images for which there are no score-able regions.

## Examples

In [19]:
%%bash
python MaskScorer.py -t manipulation --refDir ../../data/test_suite/maskScorerTests \
-r reference/manipulation/NC2017-manipulation-ref.csv -x indexes/NC2017-manipulation-index.csv \
-s ../../data/test_suite/maskScorerTests/B_NC2017_Manipulation_ImgOnly_c-me2_1/B_NC2017_Manipulation_ImgOnly_c-me2_1.csv \
-oR outputs/maniptest -html -q "ConfidenceScore < 0.5" "ManMade==['no']"

Current query: ConfidenceScore < 0.5
Current query: ManMade==['no']


Running this code in the tools/MaskScorer directory will generate an aggregate report of the computed mask scores titled B_NC2017_Manipulation_ImgOnly_c-me2_1-mask_scores.csv and a per-image score report titled B_NC2017_Manipulation_ImgOnly_c-me2_1-mask_scores_perimage.csv for the manipulation task.

The -html flag is also set, allowing the code to generate an HTML per-image <a href="outputs/maniptest/index.html">index file</a> with the scores and metadata containing links to individual detailed reports of each image.

The user may also select which manipulation regions to score, depending on the manipulations listed under the "Purpose" column in the journalMask file. Other regions are dilated by a separate factor and counted as selective no-score zones in addition to the boundary no-score zones applied to the regions of interest. Multiple pre-filters can be applied independently to the data, resulting in the output of multiple output indices corresponding to the number of queries passed to -qm.

In [17]:
%%bash
python MaskScorer.py -t manipulation --refDir ../../data/test_suite/maskScorerTests \
-r reference/manipulation/NC2017-manipulation-ref.csv -x indexes/NC2017-manipulation-index.csv \
-s ../../data/test_suite/maskScorerTests/B_NC2017_Manipulation_ImgOnly_c-me2_1/B_NC2017_Manipulation_ImgOnly_c-me2_1.csv \
-oR outputs/maniptargets -html -qm "Purpose==['remove']" "Purpose==['clone','add']"

The sample HTML index files for the 'remove' and 'clone','add' operations can be found <a href="outputs/maniptargets/index_0/index.html">here</a> and <a href="outputs/maniptargets/index_1/index.html">here</a> respectively.

## Disclaimer

This software was developed at the National Institute of Standards
and Technology (NIST) by employees of the Federal Government in the
course of their official duties. Pursuant to Title 17 Section 105
of the United States Code, this software is not subject to copyright
protection and is in the public domain. NIST assumes no responsibility
whatsoever for use by other parties of its source code or open source
server, and makes no guarantees, expressed or implied, about its quality,
reliability, or any other characteristic.